In [10]:
import pandas as pd

df_umsatz = pd.read_csv('./umsatzdaten_gekuerzt.csv')

df_wetter = pd.read_csv('wetter.csv')

merged_df = pd.merge(df_umsatz, df_wetter, on='Datum', how='inner')
print(merged_df.describe())

df_kiwo = pd.read_csv('kiwo.csv')
merged_df_with_kiwo = pd.merge(merged_df, df_kiwo, on='Datum', how='left')

print(merged_df_with_kiwo.head())


       Warengruppe       Umsatz   Bewoelkung   Temperatur  \
count  9318.000000  9318.000000  9264.000000  9318.000000   
mean      3.088002   206.782933     4.720747    12.028483   
std       1.488889   144.586222     2.646273     7.232829   
min       1.000000     7.051201     0.000000    -8.475000   
25%       2.000000    96.864379     3.000000     6.237500   
50%       3.000000   161.945865     6.000000    11.625000   
75%       4.000000   280.692490     7.000000    17.962500   
max       6.000000  1879.461831     8.000000    31.437500   

       Windgeschwindigkeit   Wettercode  
count          9318.000000  7009.000000  
mean             10.974780    36.900128  
std               4.131974    27.095884  
min               3.000000     0.000000  
25%               8.000000    10.000000  
50%              10.000000    28.000000  
75%              13.000000    61.000000  
max              35.000000    95.000000  
        Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0  201